# 한글-Claude-v2 Model: Invoke Bedrock model for text generation using zero-shot prompt
---

### 중요
- 이 노트북은 Anthropic 의 Claude-v2 모델 접근 가능한 분만 실행 가능합니다. 
- 접근이 안되시는 분은 노트북의 코드와 결과 만을 확인 하시면 좋겠습니다.
- 만일 실행시에는 **"과금"** 이 발생이 되는 부분 유념 해주시기 바랍니다.

#### 사용 사례
Amazon Bedrock에서 모델의 생성 기능을 시연하기 위해 이메일 생성 사용 사례를 살펴보겠습니다.

#### 페르소나
귀하는 AnyCompany의 고객 서비스 관리자인 권율이며 일부 고객은 고객 서비스에 만족하지 않고 고객 지원 엔지니어가 제공하는 서비스에 대해 부정적인 피드백을 제공하고 있습니다. 이제는 열악한 서비스에 대해 겸허히 사과하는 고객에게 응답하고 신뢰를 회복하고 싶습니다. 인간 친화적이고 이전 이메일 서신에서 고객의 정서에 맞게 개인화된 대량의 이메일을 생성하려면 LLM의 도움이 필요합니다.

#### 구현
이 사용 사례를 이행하기 위해 이 노트북에서는 고객의 이전 이메일을 기반으로 감사 메모가 포함된 이메일을 생성하는 방법을 보여줍니다. Boto3 클라이언트와 함께 Amazon Bedrock API를 사용하는 Amazon Titan Text Large 모델을 사용합니다.

## 1. Bedrock Client 생성

In [1]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

Create new client
  Using region: None
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)


## 2.프롬프트 생성

위에서 설명한 사용 사례에 따라 이메일을 생성하기 위해 Amazon Bedrock 서비스에 대한 입력을 준비하겠습니다.

In [4]:
# 한글
prompt_data = """
명령: 고객 서비스 관리자인 권율이 고객 "이순신"에게 이메일을 작성합니다.
이순신님은 고객 지원 엔지니어로 부터 제공하는 서비스에 대해 부정적인 피드백을 받았습니다.
"""



## 3.모델 파라미터 생성

다음은 Anthropic Claude 의 입력/출력 구조 입니다.

### Anthropic Claude

#### Input

```json
{
    "prompt": "\n\nHuman:<prompt>\n\nAssistant:",
    "max_tokens_to_sample": 300,
    "temperature": 0.5,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman:"]
}
```

#### Output

```json
{
    "completion": "<output>",
    "stop_reason": "stop_sequence"
}
```

In [5]:
# Claude Prompt
body = json.dumps({
    "prompt": f"\n\nHuman:{prompt_data}\n\nAssistant:",
    "max_tokens_to_sample": 500,
    "temperature": 0.5,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman:"]            
    }) 


## 4.invoke_model 함수 호출

Amazon Bedrock API는 다음을 허용하는 API `invoke_model`을 제공합니다.
- `modelId`: Amazon Bedrock에서 사용할 수 있는 다양한 기반 모델의 모델 ARN입니다.
- `accept`: 입력 요청 유형
- `contentType`: 출력의 콘텐츠 유형
- `body`: 프롬프트와 구성으로 구성된 json 문자열


In [6]:
modelId = 'anthropic.claude-v2'
accept = 'application/json'
contentType = 'application/json'

response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

outputText = response_body.get("completion")
print(outputText)

 안녕하세요 이순신님, 

저는 고객서비스 관리자 권율입니다. 

고객님께서 최근 저희 서비스에 대한 부정적인 피드백을 주셨다는 소식을 듣고 깊이 유감스럽게 생각합니다. 

고객님의 의견은 저희 서비스 개선에 매우 소중합니다. 불편하셨던 점이 무엇이었는지 구체적으로 알려주시면, 저희 서비스 및 직원 교육을 개선하는데 반영하겠습니다. 

앞으로 저희 서비스가 고객님의 기대에 부응할 수 있도록 최선을 다하겠습니다. 

다시 한번 지적해 주신 점에 대해 깊이 사과드리며, 고객님의 의견에 귀 기울이겠습니다. 

좋은 하루 되세요.

감사합니다.

권율 드림


## 5. 결론
이제 Amazon Bedrock API에 `boto3` SDK를 사용하여 실험했습니다. 이 API를 사용하여 부정적인 피드백으로 인해 고객에게 응답하는 이메일을 생성하는 사용 사례를 확인했습니다.

### 다음 액션
- 프롬프트를 특정 사용 사례로 변경하고 다른 모델의 출력을 평가합니다.
- 서비스의 대기 시간과 응답성을 이해하기 위해 토큰 길이를 가지고 놀아보세요.
- 더 나은 결과를 얻기 위해 다양한 프롬프트 엔지니어링 원칙을 적용합니다.

